In [1]:
import pandas as pd
import os 

team_filename = 'game_summary_by_team.csv'
data_dir = os.getcwd().replace('/notebooks', '') + '/data/external'
path = os.path.join(data_dir, team_filename)

df = pd.read_csv(path)
df.head()

,url,season,week,team,opponent,home_flg,favorite_flg,winner_flg,line,over_under,...,team_yds,team_tds,team_fgs,team_punts,opp_drives,opp_plays,opp_yds,opp_tds,opp_fgs,opp_punts
0,/boxscores/201209050nyg.htm,2012,1,Dallas Cowboys,New York Giants,False,False,True,-4.0,46.0,...,368,3,1,2,9,55,269,2,1,4
1,/boxscores/201209050nyg.htm,2012,1,New York Giants,Dallas Cowboys,True,True,False,4.0,46.0,...,269,2,1,4,9,58,368,3,1,2
2,/boxscores/201209090chi.htm,2012,1,Chicago Bears,Indianapolis Colts,True,True,True,10.0,42.5,...,415,5,2,5,13,64,372,2,0,5
3,/boxscores/201209090chi.htm,2012,1,Indianapolis Colts,Chicago Bears,False,False,False,-10.0,42.5,...,372,2,0,5,15,72,415,5,2,5
4,/boxscores/201209090nyj.htm,2012,1,New York Jets,Buffalo Bills,True,True,True,3.0,39.0,...,364,4,2,2,12,58,370,4,0,2


In [4]:
#df["line"] = df["line"].astype(float) * -1
df.head()

,url,season,week,team,opponent,home_flg,favorite_flg,winner_flg,line,over_under,...,team_yds,team_tds,team_fgs,team_punts,opp_drives,opp_plays,opp_yds,opp_tds,opp_fgs,opp_punts
0,/boxscores/201209050nyg.htm,2012,1,Dallas Cowboys,New York Giants,False,False,True,4.0,46.0,...,368,3,1,2,9,55,269,2,1,4
1,/boxscores/201209050nyg.htm,2012,1,New York Giants,Dallas Cowboys,True,True,False,-4.0,46.0,...,269,2,1,4,9,58,368,3,1,2
2,/boxscores/201209090chi.htm,2012,1,Chicago Bears,Indianapolis Colts,True,True,True,-10.0,42.5,...,415,5,2,5,13,64,372,2,0,5
3,/boxscores/201209090chi.htm,2012,1,Indianapolis Colts,Chicago Bears,False,False,False,10.0,42.5,...,372,2,0,5,15,72,415,5,2,5
4,/boxscores/201209090nyj.htm,2012,1,New York Jets,Buffalo Bills,True,True,True,-3.0,39.0,...,364,4,2,2,12,58,370,4,0,2


In [3]:
df.to_csv(path, index=False)

,url,season,week,team,opponent,home_flg,favorite_flg,winner_flg,line,over_under,...,team_yds,team_tds,team_fgs,team_punts,opp_drives,opp_plays,opp_yds,opp_tds,opp_fgs,opp_punts
0,/boxscores/201209050nyg.htm,2012,1,Dallas Cowboys,New York Giants,False,False,True,4.0,46.0,...,368,3,1,2,9,55,269,2,1,4
1,/boxscores/201209050nyg.htm,2012,1,New York Giants,Dallas Cowboys,True,True,False,-4.0,46.0,...,269,2,1,4,9,58,368,3,1,2
2,/boxscores/201209090chi.htm,2012,1,Chicago Bears,Indianapolis Colts,True,True,True,-10.0,42.5,...,415,5,2,5,13,64,372,2,0,5
3,/boxscores/201209090chi.htm,2012,1,Indianapolis Colts,Chicago Bears,False,False,False,10.0,42.5,...,372,2,0,5,15,72,415,5,2,5
4,/boxscores/201209090nyj.htm,2012,1,New York Jets,Buffalo Bills,True,True,True,-3.0,39.0,...,364,4,2,2,12,58,370,4,0,2
